In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [8]:
df= pd.read_csv(r"C:\Users\pedro\OneDrive\Desktop\DSProject Folder\src\Financial Data 14-18\Int DFs\df_mod.csv")

In [9]:
df.shape

(19718, 26)

In [10]:
df.head(10)

,Effect of forex changes on cash,SG&A Expenses Growth,Receivables Turnover,5Y Revenue Growth (per Share),3Y Shareholders Equity Growth (per Share),Issuance (buybacks) of shares,eBITperRevenue,Net cash flow / Change in cash,priceSalesRatio,SG&A to Revenue,...,Operating Cash Flow,3Y Net Income Growth (per Share),returnOnCapitalEmployed,Tangible Book Value per Share,Investing Cash flow,Gross Margin,PFCF ratio,5Y Operating CF Growth (per Share),Class,Following Year Price Variation [%]
0,-1.564869e+07,1.73130,6.48354,0.06600,0.79730,1.767840e+06,0.050221,4.463169e+08,0.095727,0.0922,...,5.267456e+08,0.25206,0.0000,4.49390,-6.866936e+08,0.2487,1.3589,0.11996,0,-25.512193
1,0.000000e+00,0.02340,90.79370,0.10380,0.07890,-4.130000e+08,0.027578,1.630000e+08,0.000000,0.1545,...,3.573000e+09,0.18920,0.0859,25.72400,-4.771000e+09,0.2057,14.6302,0.09370,1,33.118297
2,0.000000e+00,-0.00600,27.17690,-0.02900,0.00000,3.321700e+07,0.026436,1.695400e+07,0.000000,0.2570,...,7.020460e+08,0.00000,0.1062,134.78500,-3.649240e+08,0.2869,17.2736,0.11640,1,2.752291
3,-2.920000e+07,-0.02200,12.22500,0.05670,0.02170,-1.637200e+09,0.168072,1.259000e+08,1.553911,0.1940,...,2.541000e+09,0.01770,0.1041,15.42900,-5.618000e+08,0.3557,17.6902,0.08280,1,12.897715
4,-3.760000e+08,0.01610,20.39100,0.09610,0.00000,-3.833000e+09,0.145332,-4.720000e+08,1.299472,0.0874,...,7.739000e+09,-0.00840,0.3752,15.32700,-9.960000e+08,0.2413,19.2150,0.03770,1,13.980937
5,-2.230000e+08,-0.02560,6.74490,-0.04940,-0.06070,-1.700000e+09,0.094177,-9.910000e+08,1.644111,0.2470,...,3.562000e+09,-0.13540,0.0376,13.71900,-1.642000e+09,0.3679,31.6718,-0.09250,1,23.809818
6,0.000000e+00,0.08500,205.20500,0.01770,-0.05190,-9.390000e+08,0.349971,1.460000e+08,3.319691,0.1035,...,4.663000e+09,0.15880,0.1948,8.67300,1.770000e+08,0.4143,21.6402,0.07260,1,23.865489
7,3.788960e+07,-0.00760,11.93360,0.15340,0.09406,3.131806e+07,0.408287,-7.570785e+08,2.252915,0.2883,...,7.052460e+09,0.05698,0.2515,0.98010,-1.790462e+09,0.6635,6.7470,0.19388,0,-22.605312
8,6.943980e+05,0.34600,0.00000,0.45210,0.12170,0.000000e+00,0.214357,8.484990e+07,0.000000,0.3379,...,1.015584e+08,0.28460,0.0000,0.88800,-1.740287e+07,0.5172,6.8263,0.27070,1,65.373665
9,-1.320000e+08,-0.01442,15.78346,0.03482,0.07494,0.000000e+00,0.134957,-1.610000e+08,12.993151,0.1565,...,2.140000e+09,0.03054,0.0433,20.10842,-3.490000e+08,0.3000,22.0915,0.04848,1,2.065963
